## 1. Load the data

In [1]:
import pandas as pd

# Format scientific notation from Pandas
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', None)

INPUT_CSV = "/home/sramkova/diploma_thesis_data/pca_with_labels_backup_16_10_2021_9b.csv"

data = pd.read_csv(INPUT_CSV, low_memory=False)
df = data.copy()
df.head(5)

,originated_ip_num,responded_ip_num,connection.time,connection.duration,connection.orig_bytes,connection.orig_pkts,connection.orig_ip_bytes,connection.resp_bytes,connection.resp_pkts,connection.resp_ip_bytes,orig_p_cat,resp_p_cat,protocol,service,conn_state,dns_count,ssh_count,http_count,ssl_count,files_count,orig_orig_total,orig_orig_connection.time_mean,orig_orig_connection.duration_mean,orig_orig_connection.orig_pkts_mean,orig_orig_connection.orig_bytes_mean,orig_orig_connection.resp_bytes_mean,orig_orig_connection.resp_pkts_mean,orig_orig_orig_p_well_known_count,orig_orig_orig_p_reg_or_dyn_count,orig_orig_resp_p_21_count,orig_orig_resp_p_22_count,orig_orig_resp_p_53_count,orig_orig_resp_p_80_count,orig_orig_resp_p_123_count,orig_orig_resp_p_443_count,orig_orig_resp_p_3389_count,orig_orig_resp_p_well_known_count,orig_orig_resp_p_reg_count,orig_orig_resp_p_dyn_count,orig_orig_dns_count_mean,orig_orig_ssh_count_mean,orig_orig_http_count_mean,orig_orig_ssl_count_mean,orig_orig_files_count_mean,orig_orig_similar_conns_count,orig_orig_similar_dns_qtype_count,orig_orig_similar_dns_rcode_count,orig_orig_similar_ssh_auth_attempts_count,orig_orig_similar_ssh_host_key_count,orig_orig_similar_http_method_count,orig_orig_similar_http_status_code_count,orig_orig_similar_http_user_agent_count,orig_orig_similar_ssl_version_count,orig_orig_similar_ssl_cipher_count,orig_orig_similar_ssl_curve_count,orig_orig_similar_ssl_validation_status_count,orig_orig_similar_files_source_count,orig_orig_similar_file_md5_count,orig_orig_connection.protocol_mode,orig_orig_connection.service_mode,orig_orig_connection.conn_state_mode,orig_resp_total,orig_resp_connection.time_mean,orig_resp_connection.duration_mean,orig_resp_connection.orig_pkts_mean,orig_resp_connection.orig_bytes_mean,orig_resp_connection.resp_bytes_mean,orig_resp_connection.resp_pkts_mean,orig_resp_orig_p_well_known_count,orig_resp_orig_p_reg_or_dyn_count,orig_resp_resp_p_21_count,orig_resp_resp_p_22_count,orig_resp_resp_p_53_count,orig_resp_resp_p_80_count,orig_resp_resp_p_123_count,orig_resp_resp_p_443_count,orig_resp_resp_p_3389_count,orig_resp_resp_p_well_known_count,orig_resp_resp_p_reg_count,orig_resp_resp_p_dyn_count,orig_resp_dns_count_mean,orig_resp_ssh_count_mean,orig_resp_http_count_mean,orig_resp_ssl_count_mean,orig_resp_files_count_mean,orig_resp_similar_conns_count,orig_resp_similar_dns_qtype_count,orig_resp_similar_dns_rcode_count,orig_resp_similar_ssh_auth_attempts_count,orig_resp_similar_ssh_host_key_count,orig_resp_similar_http_method_count,orig_resp_similar_http_status_code_count,orig_resp_similar_http_user_agent_count,orig_resp_similar_ssl_version_count,orig_resp_similar_ssl_cipher_count,orig_resp_similar_ssl_curve_count,orig_resp_similar_ssl_validation_status_count,orig_resp_similar_files_source_count,orig_resp_similar_file_md5_count,orig_resp_connection.protocol_mode,orig_resp_connection.service_mode,orig_resp_connection.conn_state_mode,resp_orig_total,resp_orig_connection.time_mean,resp_orig_connection.duration_mean,resp_orig_connection.orig_pkts_mean,resp_orig_connection.orig_bytes_mean,resp_orig_connection.resp_bytes_mean,resp_orig_connection.resp_pkts_mean,resp_orig_orig_p_well_known_count,resp_orig_orig_p_reg_or_dyn_count,resp_orig_resp_p_21_count,resp_orig_resp_p_22_count,resp_orig_resp_p_53_count,resp_orig_resp_p_80_count,resp_orig_resp_p_123_count,resp_orig_resp_p_443_count,resp_orig_resp_p_3389_count,resp_orig_resp_p_well_known_count,resp_orig_resp_p_reg_count,resp_orig_resp_p_dyn_count,resp_orig_dns_count_mean,resp_orig_ssh_count_mean,resp_orig_http_count_mean,resp_orig_ssl_count_mean,resp_orig_files_count_mean,resp_orig_similar_conns_count,resp_orig_similar_dns_qtype_count,resp_orig_similar_dns_rcode_count,resp_orig_similar_ssh_auth_attempts_count,resp_orig_similar_ssh_host_key_count,resp_orig_similar_http_method_count,resp_orig_similar_http_status_code_count,resp_orig_similar_http_user_agent_count,resp_orig_similar_ssl_version_count,resp_orig_similar_ssl_c

In [2]:
GAMMA = 0.08900004897193284

In [3]:
#df = df.sample(1000)

In [4]:
df['cluster'].value_counts()

3    88942
0    78371
6    77087
7    52301
1    51323
9    41711
2    41274
8    19886
4    11993
5     9457
Name: cluster, dtype: int64

### Silhouette coefficient evaluation

```
Silhouette Score = (average distance between points within a cluster - average distance between all clusters) / 
                   max(average distance between all clusters, average distance between points within a cluster)

```

- intRA-cluster distance = average distance between each point within a cluster
- intER-cluster distance = average distance between all clusters

<center>
<img src="https://latex.codecogs.com/gif.latex?\bg_white&space;SilhouetteScore(dataPoint) = \frac{b(dataPoint) - a(dataPoint)}{max\{a(dataPoint),b(dataPoint)\}}" /></a>
</center>

<img src="https://latex.codecogs.com/svg.latex?\Large&space;SilhouetteScore(dataPoint) = \frac{b(dataPoint) - a(dataPoint)}{max\{a(dataPoint),b(dataPoint)\}}" />

<center>
<img src="https://latex.codecogs.com/gif.latex?\bg_white&space;a(dataPoint) = \frac{1}{|C_i| - 1} * \sum_{j \in C_i, i != j} d({i,j)}" /></a>
</center>

<img src="https://latex.codecogs.com/svg.latex?\Large&space;a(dataPoint) = \frac{1}{|C_i| - 1} * \sum_{j \in C_i, i != j} d({i,j)}" />

<center>
<img src="https://latex.codecogs.com/gif.latex?\bg_white&space;b(dataPoint) = min_{k != i} \frac{1}{|C_k|} * \sum_{j \in C_k} d({i,j)}" /></a>
</center>

<img src="https://latex.codecogs.com/svg.latex?\Large&space;b(dataPoint) = min_{k != i} \frac{1}{|C_k|} * \sum_{j \in C_k} d({i,j)}" />

In [5]:
len(df.columns)

186

In [6]:
# Inspect the categorical variables
df.select_dtypes('object').nunique()

orig_p_cat                               2
resp_p_cat                              10
protocol                                 3
service                                 14
conn_state                              13
orig_orig_connection.protocol_mode       3
orig_orig_connection.service_mode        8
orig_orig_connection.conn_state_mode    12
orig_resp_connection.protocol_mode       4
orig_resp_connection.service_mode        9
orig_resp_connection.conn_state_mode    12
resp_orig_connection.protocol_mode       4
resp_orig_connection.service_mode        8
resp_orig_connection.conn_state_mode    12
resp_resp_connection.protocol_mode       3
resp_resp_connection.service_mode        8
resp_resp_connection.conn_state_mode    12
dtype: int64

In [7]:
# Get the position of categorical columns
cat_cols_i_list = [df.columns.get_loc(col) for col in list(df.select_dtypes('object').columns)]
print('Categorical columns         : {}'.format(list(df.select_dtypes('object').columns)))
print('Categorical columns indexes : {}'.format(cat_cols_i_list))

Categorical columns         : ['orig_p_cat', 'resp_p_cat', 'protocol', 'service', 'conn_state', 'orig_orig_connection.protocol_mode', 'orig_orig_connection.service_mode', 'orig_orig_connection.conn_state_mode', 'orig_resp_connection.protocol_mode', 'orig_resp_connection.service_mode', 'orig_resp_connection.conn_state_mode', 'resp_orig_connection.protocol_mode', 'resp_orig_connection.service_mode', 'resp_orig_connection.conn_state_mode', 'resp_resp_connection.protocol_mode', 'resp_resp_connection.service_mode', 'resp_resp_connection.conn_state_mode']
Categorical columns indexes : [10, 11, 12, 13, 14, 58, 59, 60, 99, 100, 101, 140, 141, 142, 181, 182, 183]


In [8]:
pd.set_option('display.max_columns', None)

In [9]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.metrics.pairwise import euclidean_distances
# from kmodes.kprototypes import KPrototypes
from kmodes.util.dissim import matching_dissim
from kmodes.util.dissim import euclidean_dissim
from kmodes.util import encode_features
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [10]:
# cat_cols_i_list
num_df = df.drop(list(df.select_dtypes('object').columns)+['cluster','cluster_cat'], axis=1)
cat_df = df[list(df.select_dtypes('object').columns)]
label_df = df['cluster']

In [11]:
from datetime import datetime

In [12]:
# cat_cols_i_list
comp_num_df = df.drop(list(df.select_dtypes('object').columns) + ['cluster_cat'], axis=1)
comp_cat_df = df[list(df.select_dtypes('object').columns) + ['cluster']]

In [ ]:
# https://github.com/nicodv/kmodes/blob/master/kmodes/util/dissim.py
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html

In [13]:
def compute_ai_d(comp_num_df, comp_cat_df, gamma):
    # encode categories to numbers:
    LE = LabelEncoder()
    for cat_feature in comp_cat_df.columns:
        comp_cat_df[cat_feature] = LE.fit_transform(comp_cat_df[cat_feature])
    
    comp_num_df['ai_d'] = 0.0
    cluster_numbers = comp_num_df['cluster'].unique()
    
    for cluster_number in cluster_numbers:
        # for cluster C_i compute d(i,j) in a(i) formula:
        clu_num_df = comp_num_df[comp_num_df['cluster'] == cluster_number]
        clu_cat_df = comp_cat_df[comp_cat_df['cluster'] == cluster_number]
        
        # choose current data point i (row):
        for row in clu_num_df.iterrows():
            row_index = row[0]
            row_cat = clu_cat_df.loc[[row_index]] ## TODO

            # drop current row
            clu_num_df_array = clu_num_df.drop(row_index).to_numpy()
            clu_cat_df_array = clu_cat_df.drop(row_index).to_numpy() 

            # get Euclidean distances of current row to all other rows from the same cluster and sum it 
            inside_cluster_distances_num = np.sum(euclidean_distances([row[1]], clu_num_df_array))
            
            # get matching dissimilarity
            inside_cluster_distances_cat = np.sum(row_cat.to_numpy() != clu_cat_df_array)
            
            # save the subresult to df
            comp_num_df.at[row_index, 'ai_num_d'] = (1 / (len(clu_num_df.index) - 1)) * (inside_cluster_distances_num + gamma * inside_cluster_distances_cat)

print('Start at ' + datetime.now().strftime("%H:%M:%S") + '.')            
compute_ai_d(comp_num_df, comp_cat_df, GAMMA)
print('Done at ' + datetime.now().strftime("%H:%M:%S") + '.')

Start at 09:59:24.


<ipython-input-13-6a55de61091f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_cat_df[cat_feature] = LE.fit_transform(comp_cat_df[cat_feature])
<ipython-input-13-6a55de61091f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_cat_df['ai_cat_d'] = 0.0


Done at 03:32:03.


In [14]:
comp_num_df

,originated_ip_num,responded_ip_num,connection.time,connection.duration,connection.orig_bytes,connection.orig_pkts,connection.orig_ip_bytes,connection.resp_bytes,connection.resp_pkts,connection.resp_ip_bytes,dns_count,ssh_count,http_count,ssl_count,files_count,orig_orig_total,orig_orig_connection.time_mean,orig_orig_connection.duration_mean,orig_orig_connection.orig_pkts_mean,orig_orig_connection.orig_bytes_mean,orig_orig_connection.resp_bytes_mean,orig_orig_connection.resp_pkts_mean,orig_orig_orig_p_well_known_count,orig_orig_orig_p_reg_or_dyn_count,orig_orig_resp_p_21_count,orig_orig_resp_p_22_count,orig_orig_resp_p_53_count,orig_orig_resp_p_80_count,orig_orig_resp_p_123_count,orig_orig_resp_p_443_count,orig_orig_resp_p_3389_count,orig_orig_resp_p_well_known_count,orig_orig_resp_p_reg_count,orig_orig_resp_p_dyn_count,orig_orig_dns_count_mean,orig_orig_ssh_count_mean,orig_orig_http_count_mean,orig_orig_ssl_count_mean,orig_orig_files_count_mean,orig_orig_similar_conns_count,orig_orig_similar_dns_qtype_count,orig_orig_similar_dns_rcode_count,orig_orig_similar_ssh_auth_attempts_count,orig_orig_similar_ssh_host_key_count,orig_orig_similar_http_method_count,orig_orig_similar_http_status_code_count,orig_orig_similar_http_user_agent_count,orig_orig_similar_ssl_version_count,orig_orig_similar_ssl_cipher_count,orig_orig_similar_ssl_curve_count,orig_orig_similar_ssl_validation_status_count,orig_orig_similar_files_source_count,orig_orig_similar_file_md5_count,orig_resp_total,orig_resp_connection.time_mean,orig_resp_connection.duration_mean,orig_resp_connection.orig_pkts_mean,orig_resp_connection.orig_bytes_mean,orig_resp_connection.resp_bytes_mean,orig_resp_connection.resp_pkts_mean,orig_resp_orig_p_well_known_count,orig_resp_orig_p_reg_or_dyn_count,orig_resp_resp_p_21_count,orig_resp_resp_p_22_count,orig_resp_resp_p_53_count,orig_resp_resp_p_80_count,orig_resp_resp_p_123_count,orig_resp_resp_p_443_count,orig_resp_resp_p_3389_count,orig_resp_resp_p_well_known_count,orig_resp_resp_p_reg_count,orig_resp_resp_p_dyn_count,orig_resp_dns_count_mean,orig_resp_ssh_count_mean,orig_resp_http_count_mean,orig_resp_ssl_count_mean,orig_resp_files_count_mean,orig_resp_similar_conns_count,orig_resp_similar_dns_qtype_count,orig_resp_similar_dns_rcode_count,orig_resp_similar_ssh_auth_attempts_count,orig_resp_similar_ssh_host_key_count,orig_resp_similar_http_method_count,orig_resp_similar_http_status_code_count,orig_resp_similar_http_user_agent_count,orig_resp_similar_ssl_version_count,orig_resp_similar_ssl_cipher_count,orig_resp_similar_ssl_curve_count,orig_resp_similar_ssl_validation_status_count,orig_resp_similar_files_source_count,orig_resp_similar_file_md5_count,resp_orig_total,resp_orig_connection.time_mean,resp_orig_connection.duration_mean,resp_orig_connection.orig_pkts_mean,resp_orig_connection.orig_bytes_mean,resp_orig_connection.resp_bytes_mean,resp_orig_connection.resp_pkts_mean,resp_orig_orig_p_well_known_count,resp_orig_orig_p_reg_or_dyn_count,resp_orig_resp_p_21_count,resp_orig_resp_p_22_count,resp_orig_resp_p_53_count,resp_orig_resp_p_80_count,resp_orig_resp_p_123_count,resp_orig_resp_p_443_count,resp_orig_resp_p_3389_count,resp_orig_resp_p_well_known_count,resp_orig_resp_p_reg_count,resp_orig_resp_p_dyn_count,resp_orig_dns_count_mean,resp_orig_ssh_count_mean,resp_orig_http_count_mean,resp_orig_ssl_count_mean,resp_orig_files_count_mean,resp_orig_similar_conns_count,resp_orig_similar_dns_qtype_count,resp_orig_similar_dns_rcode_count,resp_orig_similar_ssh_auth_attempts_count,resp_orig_similar_ssh_host_key_count,resp_orig_similar_http_method_count,resp_orig_similar_http_status_code_count,resp_orig_similar_http_user_agent_count,resp_orig_similar_ssl_version_count,resp_orig_similar_ssl_cipher_count,resp_orig_similar_ssl_curve_count,resp_orig_similar_ssl_validation_status_count,resp_orig_similar_files_source_count,resp_orig_similar_file_md5_count,resp_resp_total,resp_resp_connection.time_mean,resp_resp_connection.duration_mean,resp_resp_connecti

In [ ]:
def compute_bi_d(comp_num_df, comp_cat_df, gamma):
    # encode categories to numbers:
    LE = LabelEncoder()
    for cat_feature in comp_cat_df.columns:
        comp_cat_df[cat_feature] = LE.fit_transform(comp_cat_df[cat_feature])
    
    comp_num_df['bi_num_d'] = 999.0
    comp_cat_df['bi_cat_d'] = 999.0
    cluster_numbers = comp_num_df['cluster'].unique()
    
    # for data point i from cluster C_i:
    for cluster_number in cluster_numbers:
        for cluster_number2 in cluster_numbers:
            if cluster_number == cluster_number2:
                continue
            # for cluster C_k compute d(i,j) in b(i) formula:
            clu_num_df = comp_num_df[comp_num_df['cluster'] == cluster_number2]
            clu_cat_df = comp_cat_df[comp_cat_df['cluster'] == cluster_number2]

            # choose current data point i (row):
            for row in clu_num_df.iterrows():
                row_index = row[0]
                row_cat = clu_cat_df.loc[[row_index]]

                # drop current row
                clu_num_df_array = clu_num_df.drop(row_index).to_numpy()
                clu_cat_df_array = clu_cat_df.drop(row_index).to_numpy() 

                # get Euclidean distances of current row to all other rows from the same cluster and sum it 
                inside_cluster_distances_num = np.sum(euclidean_distances([row[1]], clu_num_df_array))

                # get matching dissimilarity
                inside_cluster_distances_cat = np.sum(row_cat.to_numpy() != clu_cat_df_array)

                # save the subresult to df
                subresult = (1 / len(clu_num_df.index)) * (inside_cluster_distances_num + gamma * inside_cluster_distances_cat)
                if comp_num_df.at[row_index, 'bi_num_d'] > subresult:
                    comp_num_df.at[row_index, 'bi_num_d'] = subresult

print('Start at ' + datetime.now().strftime("%H:%M:%S") + '.')                    
compute_bi_d(comp_num_df, comp_cat_df, GAMMA)
print('Done at ' + datetime.now().strftime("%H:%M:%S") + '.')

Start at 03:32:03.


<ipython-input-15-43d1c0ec386f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_cat_df[cat_feature] = LE.fit_transform(comp_cat_df[cat_feature])
<ipython-input-15-43d1c0ec386f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_cat_df['bi_cat_d'] = 999.0


In [ ]:
comp_num_df.to_csv('/home/sramkova/diploma_thesis_data/sil_backup_comp_df.csv', index=False, header=True)

In [ ]:
comp_num_df

In [ ]:
comp_num_df['sil_result'] = (comp_num_df['bi_num_d'] - comp_num_df['ai_num_d']) / comp_num_df[['ai_num_d', 'bi_num_d']].values.max(1)
comp_num_df

In [ ]:
comp_num_df['sil_result'].mean()

In [ ]:
comp_num_df.to_csv('/home/sramkova/diploma_thesis_data/sil_backup_comp_df_final.csv', index=False, header=True)